# Part I. One-sided finite differences

Write a function, `deriv`, which computes a derivative of its argument at a given point, $x$, using a one-sided finite difference rule with a given step side $h$, with the approximation order of $O(h^2)$. 

We want order of $O(h^2)$. Write one-sided difference with number of uncertain coefficients, that provides us to "kill" all powers of $h$ below needed order.
$$
\frac{df(x)}{dx} = \frac{Af(x)+Bf(x+h)+Cf(x+2h)}{h}
$$
Solution is $A = -\frac{3}{2}, B = 2, C = -\frac{1}{2}$.

In [23]:
def deriv(f, x, h):
    """ Compute a derivative of `f` at point `x` with step size `h`.
    
    Compute the derivative using the one-sided rule of the approximation order of $O(h^2)$.
    
    Parameters
    ----------
    f : callable
        The function to differentiate
    x : float
        The point to compute the derivative at.
    h : float
        The step size for the finite different rule.
        
    Returns
    -------
    fder : derivative of f(x) at point x using the step size h.
    """
    
    df = (-3/2*f(x)+2*f(x+h)-1/2*f(x+2*h))/h
    return df
    
    # ... ENTER YOUR CODE HERE ...

#### Test I.1

Test your function on a simple test case: differentiate $f(x) = x^3$ at $x=0$. Comment on whether your results are consistent with the expected value of $f'(x) = 0$ and on an expected scaling with $h\to 0$.

 (10% of the total grade)

In [24]:
x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    err = deriv(lambda x: x**3, x, h)
    print("%5f -- %7.4g" % (h, err))

0.010000 -- -0.0002
0.001000 --  -2e-06
0.000100 --  -2e-08
0.000010 --  -2e-10


We expect that correct algoritm gives out derivative that approximates true derivative when $h\to 0$. We can see higher expected.

### Test I.2

Now use a slightly more complicated function, $f(x) = x^2 \log{x}$, evaluate the derivative at $x=1$ using your one-sided rule and a two-point one-sided rule. Roughly estimate the value of $h$ where the error stops decreasing, for these two schemes. 
(15% of the total grade)

In [25]:
from math import log

def f(x):
    return x**2 * log(x)
    
def fder(x):
    return x * (2.*log(x) + 1)

def deriv1(f, x, h):
    df = (f(x+h)-f(x))/h
    return df

In [26]:
x = 1
for h in [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]:
    err = fder(x) - deriv(f, x, h)
    print("%7g -- %7.4g" % (h, err))

print('')

for h in [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]:
    err = fder(x) - deriv1(f, x, h)
    print("%7g -- %7.4g" % (h, err))

   0.01 -- 6.617e-05
  0.001 -- 6.662e-07
 0.0001 -- 6.666e-09
  1e-05 -- 4.901e-11
  1e-06 -- 1.94e-10
  1e-07 -- -1.694e-09
  1e-08 -- 1.718e-08
  1e-09 -- -1.938e-07
  1e-10 -- -8.274e-08

   0.01 -- -0.01503
  0.001 -- -0.0015
 0.0001 -- -0.00015
  1e-05 -- -1.5e-05
  1e-06 -- -1.5e-06
  1e-07 -- -1.506e-07
  1e-08 -- -8.923e-09
  1e-09 -- -8.424e-08
  1e-10 -- -8.289e-08


The value of $h$ where the error stops decreasing is $\sim 10^{-5}$ for three-point one-sided rule and is $\sim 10^{-8}$ for two-point one-sided rule.

### Test I.3 

Now try differentiating $x^2 \log(x)$ at $x=0$. Use the three-point one-sided rule. Note that to evaluate the function at zero, you need to special-case this value. Check the scaling of the error with $h$, explain your results. 
(25% of the total grade)

In [27]:
def f(x):
    if x == 0:
        # the limit of $x^2 log(x)$ at $x-> 0$ is zero, even though log(x) is undefined at x=0
        return 0.0
    else:
        return x**2 * log(x)
    
def fder(x):
    if x == 0:
        return 0.0
    else:
        return x*(2*log(x) + 1)

x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]:
    err = deriv(f, x, h) - fder(x)
    print("%5f -- %7.4g" % (h, err))

0.010000 -- -0.01386
0.001000 -- -0.001386
0.000100 -- -0.0001386
0.000010 -- -1.386e-05
0.000001 -- -1.386e-06
0.000000 -- -1.386e-07
0.000000 -- -1.386e-08
0.000000 -- -1.386e-09
0.000000 -- -1.386e-10


Error scales with $ℎ$ near $x=0$ due to form of function. Near zero we have
$$
\frac{df}{dx}=\frac{2h^2ln(h)-\frac{1}{2}4h^2ln(2h)}{h}=-2ln2h \sim h,
-2ln2 = 1.386
$$

# Part II. Midpoint rule 

Write a function which computes a definite integral using the midpoint rule up to a given error, $\epsilon$. Estimate the error by comparing the estimates of the integral at $N$ and $2N$ elementary intervals. 

In [164]:
def midpoint_rule0(f, a, b, eps):
    """ Calculate the integral of f from a to b using the midpoint rule.
    
    Parameters
    ----------
    func : callable
        The function to integrate.
    a : float
        The lower limit of integration.
    b : float
        The upper limit of integration.
    eps : float
        The target accuracy of the estimate.
        
    Returns
    -------
    integral : float
        The estimate of $\int_a^b f(x) dx$.
    """
    M = 0
    M1 = 0
    M2 = 0
    epsilon = 0
    
    n = 1
    for i in range(n):
        M += f(a+(b-a)*(2*i+1)/2/n)
    M1 = M*(b-a)/n
    
    n = 2
    for i in range(n):
        M += f(a+(b-a)*(2*i+1)/2/n)
    M2 = M*(b-a)/n
    
    epsilon = M2 - M1
    
    while abs(epsilon) > eps:
        n = n*2
        for i in range(n):
            M += f(a+(b-a)*(2*i+1)/2/n)
        M1 = M*(b-a)/n
        
        epsilon = M2 - M1
        
        M2 = M1
    
    return n, M2

### Test II.1

Test your midpoint rule on a simple integral, which you can calculate by paper and pencil.

Compare the rate of convergence to the expected $O(N^{-2})$ scaling by studying the number of intervals required for a given accuracy $\epsilon$.

Compare the numerical results to the value you calculated by hand. Does the deviation agree with your estimate of the numerical error?
(20% of the total grade)


In [165]:
def f(x):
    return x**2

a = 0
b = 1
eps = [2**(-6), 2**(-10), 2**(-14), 2**(-18), 2**(-22)]
for e in eps:
    print(e, midpoint_rule0(f, a, b, e))

0.015625 (32, 0.651123046875)
0.0009765625 (512, 0.6656904220581055)
6.103515625e-05 (8192, 0.6666056327521801)
3.814697265625e-06 (131072, 0.6666628519742517)
2.384185791015625e-07 (2097152, 0.6666664282488455)


For $x^3$ scaling can be clearly seen: to encrease accuracy in $2^{4}$ we need $2^{4}$ more elementary intervals

In [166]:
def f1(x):
    return x**(2)*log(x)

a = 0
b = 1
eps = [2**(-6), 2**(-10), 2**(-14), 2**(-18), 2**(-22), 2**(-26), 2**(-30)]
for e in eps:
    print(e, midpoint_rule0(f1, a, b, e))

0.015625 (4, -0.21950047160247332)
0.0009765625 (16, -0.22205760867716576)
6.103515625e-05 (64, -0.22221202067896506)
3.814697265625e-06 (256, -0.22222158598570813)
2.384185791015625e-07 (1024, -0.22222218247870315)
1.4901161193847656e-08 (4096, -0.22222221973858422)
9.313225746154785e-10 (16384, -0.22222222206699896)


In [167]:
import math
def f1(x):
    return math.sin(x)*math.sin(21.1*x)

a = 0
b = math.pi
eps = [2**(-6), 2**(-10), 2**(-14), 2**(-18), 2**(-22), 2**(-26), 2**(-30)]
for e in eps:
    print(e, midpoint_rule0(f1, a, b, e))

0.015625 (16, 0.0020157117257719086)
0.0009765625 (32, 0.0015223674124792141)
6.103515625e-05 (128, 0.001399092716432849)
3.814697265625e-06 (512, 0.0013917952572344322)
2.384185791015625e-07 (2048, 0.0013913406891058642)
1.4901161193847656e-08 (8192, 0.0013913122845320308)
9.313225746154785e-10 (32768, 0.0013913105092688708)


### Test II.2

Now use your midpoint rule to compute the value of

$$
\int_0^1\! \frac{\sin{\sqrt{x}}}{x}\, dx
$$

up to a predefined accuracy of $\epsilon=10^{-4}$.

Note that the integral contains an integrable singularity at the lower limit. Do calculations two ways: first, do a straightforward computation; next, subtract the singularity. Compare the number of iterations required to achieve the accuracy of $\epsilon$.

(30% of the total grade)

#### II.2.1. Straightforward computation

In [168]:
exact = 1.89216613354

#<=SHOWES VALUE OF INTEGRAL AND ERROR ON EVERY STEP=>
#<=GIVES OUT NUMBER OF STEPS AND VALUE OF INTEGRAL=>
def midpoint_rule(f, a, b, eps, exact):

    M = 0
    epsilon = 0
    
    n = 1
    for i in range(n):
        M += f(a+(b-a)*(2*i+1)/2/n)
    M = M*(b-a)/n
    
    epsilon = M - exact
    
    while abs(epsilon) > eps:
        n = n*2
        for i in range(n):
            M += f(a+(b-a)*(2*i+1)/2/n)
        M = M*(b-a)/n
        
        epsilon = M - exact
        
        print(M, epsilon)
        
    return n, M

def f2(x):
    return math.sin(math.sqrt(x))/x

a = 0
b = 1
e = 1e-4
#If applied for initial function, method works too slowly
mid1 = midpoint_rule(f2, a, b, e, exact)
print(e, mid1)

2.116328003899328 0.22416187035932822
2.1190134325614407 0.22684729902144074
1.9431025077011164 0.050936374161116404
1.8623199093737304 -0.029846224166269586
1.8433988625138598 -0.0487672710261402
1.845342912885099 -0.046823220654900854
1.8531113367356558 -0.039054796804344205
1.8615965774873842 -0.0305695560526158
1.8690682906393166 -0.023097842900683396
1.8750880331415685 -0.017078100398431495
1.8797151093490077 -0.01245102419099231
1.883173476959697 -0.008992656580302905
1.885712758028877 -0.006453375511122905
1.887555460105165 -0.004610673434835011
1.8888821181927795 -0.003284015347220448
1.8898320768629746 -0.0023340566770253535
1.8905097465593221 -0.0016563869806778264
1.890991909718482 -0.0011742238215179945
1.891334341370004 -0.0008317921699960706
1.8915772231105326 -0.0005889104294674041
1.891749339646158 -0.0004167938938419713
1.8918712310956345 -0.00029490244436547464
1.8919575147330863 -0.00020861880691369628
1.8920185731708437 -0.00014756036915630766
1.8920617713561076 -0.

#### II.2.2. Subtracting the singularity

On a paper we can get that

$$
\int_0^1\! \frac{\sin{\sqrt{x}}}{x}\, dx = \frac{1}{2} + \int_0^1\! \frac{\sin{\sqrt{x}}-\sqrt{x}}{x}\, dx
$$

Compute derived integral

In [144]:
def f3(x):
    return (math.sin(math.sqrt(x))-math.sqrt(x))/x

mid2 = midpoint_rule(f3, a, b, e, exact-2)
print(e, mid2)

-0.16812904647684535 -0.06029518001684531
-0.1509449096122759 -0.043111043152275874
-0.1271032868054397 -0.01926942034543966
-0.1159246374226674 -0.00809077096266736
-0.11150963927991028 -0.0036757728199102446
-0.10959528815494225 -0.001761421694942214
-0.10869689793049024 -0.0008630314704902026
-0.10826088860633741 -0.0004270221463373719
-0.10804617090360029 -0.00021230444360025025
-0.10793967996107087 -0.00010581350107083387
-0.10788667297189697 -5.2806511896930086e-05
0.0001 (2048, -0.10788667297189697)


#### II.2.3. Transform the integral and do both straightforward computation and subtracting the singularity.

We also can get another represent of our integral

$$
\int_0^1\! \frac{\sin{\sqrt{x}}}{x}\, dx = 2\int_0^1\! \frac{\sin{x}}{x}\, dx = 2 + \int_0^1\! \frac{\sin{x}-x}{x}\, dx
$$


In [147]:
def f4(x):
    return 2*math.sin(x)/x

mid3 = midpoint_rule(f4, a, b, e, exact)
print(e, mid3)
print('')

def f5(x):
    return 2*(math.sin(x)-x)/x

mid4 = midpoint_rule(f5, a, b, e, exact-2)
print(e, mid4)

2.85731859425761 0.9651524607176101
2.6080660595644285 0.7158999260244285
2.218566650017695 0.32640051647769486
2.030924599608406 0.13875846606840625
1.9556570440524788 0.0634909105124788
1.9227294093658023 0.030563275825802316
1.9071889960697548 0.015022862529754821
1.8996164807061433 0.007450347166143345
1.8958764249121973 0.0037102913721973607
1.8940176064903065 0.0018514729503065208
1.8930909600024701 0.000924826462470163
1.892628322640448 0.0004621891004479295
1.8923971748391424 0.00023104129914242044
1.8922816435851249 0.00011551004512488916
1.892223888610648 5.7755070647980133e-05
0.0001 (32768, 1.892223888610648)

-0.14268140574239008 -0.03484753928239004
-0.14193394043557142 -0.03410007397557138
-0.12518334998230485 -0.01734948352230481
-0.11555977539159384 -0.007725908931593806
-0.11142059266627109 -0.003586726206271054
-0.10956867870792875 -0.0017348122479287076
-0.10868833274332144 -0.0008544662833214062
-0.10825804010953372 -0.00042417364953367676
-0.10804520501127195 -0.0

#### II.2.3. Subtracting the singularity and using of Taylor series (im sorry for this but i couldnt not to try)

$$
\int_0^1\! \frac{\sin{\sqrt{x}}}{x}\, dx = 2 + \int_0^1\! \frac{\sin{\sqrt{x}}-\sqrt{x}}{x}\, dx
$$

$$
\int_0^1\! \frac{\sin{\sqrt{x}}-\sqrt{x}}{x}\, dx = \int_0^1\! \frac{1}{x}(\sum_{n=0}^{\infty}(-1)^n\frac{x^{\frac{2n+1}{2}}}{(2n+1)!} - \sqrt x) \, dx = \int_0^1\! \frac{1}{x}(\sum_{n=1}^{\infty}(-1)^n\frac{x^{\frac{2n+1}{2}}}{(2n+1)!}) \, dx = 
$$

$$
\int_0^1\! \sum_{n=1}^{\infty}(-1)^n\frac{x^{\frac{2n-1}{2}}}{(2n+1)!} \, dx = \sum_{n=1}^{\infty}\frac{(-1)^n}{(2n+1)!} \int_0^1\! x^{\frac{2n-1}{2}} \, dx
$$

Evaluate intagral from above to get needed accuracy \epsilon

$$
\sum_{n=1}^{\infty}\frac{(-1)^n}{(2n+1)!} \int_0^1\! x^{\frac{2n-1}{2}} \, dx = \sum_{n=1}^{\infty}\frac{(-1)^n}{(2n+1)!} \frac{b-a}{N} \sum_{k=1}^{N} x_k^{\frac{2n-1}{2}} \leq \sum_{n=1}^{\infty}\frac{(-1)^n}{(2n+1)!} = 
$$

$$
\sum_{n=1}^{M}\frac{(-1)^n}{(2n+1)!} + \sum_{n=M+1}^{\infty}\frac{(-1)^n}{(2n+1)!} \leq \sum_{n=1}^{M}\frac{(-1)^n}{(2n+1)!} + (1 + q^2 + q^3 +...) = \sum_{n=1}^{M}\frac{(-1)^n}{(2n+1)!} + \frac{q}{1-q},
$$

and
$$
q = \frac{1}{(2M+1)!}/\frac{1}{(2M+3)!} = \frac{1}{(2M+2)(2M+3)}, \;  \epsilon = \frac{q}{1-q}
$$

where we can get upper evaluate for M
$$
M \approx 2\sqrt{(1-\epsilon)/\epsilon} \approx 200
$$

We want to see if this method allows to get accurasy by less computations comparing with above ones. I have found that lower number of calculations to deal with needed accuracy can be gotten with these parameters: $N = 21$ elementary intervals and $M = 3$ members Taoylor series in evey point.

In [188]:
def midpoint_rule1():

    S = 0
    n = 21
    M = 3
    for i in range(n):
        for j in range(M-1):
            S += ((2*i+1)/2/n)**((2*j+1)/2)*((-1)**(j+1))/math.factorial(2*j+3)/n
            
    #<=CLICK THERE TO CHECK IF ACCURASY HOLDS=>
    #print(exact-S-2)
    return S

integral = midpoint_rule1()

print('<=Number of steps -- value of integral=>')
print(mid1[0], mid1[1])
print(mid2[0], 2 + mid2[1])
print(mid3[0], mid3[1])
print(mid4[0], 2 + mid4[1])
print(63, 2+integral)

<=Number of steps -- value of integral=>
67108864 1.8920923287613507
2048 1.892113327028103
32768 1.892223888610648
2048 1.892113441958174
63 1.8921235302348418


### III. Сonclusion:
#1) Straightforward computatuion of integrals of functions sigular on integration segment is costly and should be avoided
#2) Subtracting the singularity make such integrals more "cheeper"
#3) Transforming of integral to another if it is possible also can make integral "cheeper"
#4) More detailed transforming of integral "on paper" can be more effective than just ubtracting the singularity